<img src = "logo_ensae.png" style="float:right;width:30%"> </img>
<p style="font-size:2em;">Parallel MCMC </p>
<p style="font-size:1.5em">Eléments logiciels pour le traitement des données massives </p>
http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/projet_info_3A.html

Raphaël Huille & Michael Sok <br>
ENSAE ParisTech

Nous avons choisi d'implémenter l'algorithme Independent Metropolis–Hastings (IMH) décrit dans l'article : 
[*Using parallel computation to improve Independent Metropolis–Hastings based estimation* (2011)](https://arxiv.org/pdf/1010.1595v3.pdf) en utilisant la fonction *Pool* du pkg *multiprocessing*

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
import matplotlib.pyplot as plt
from time import time, sleep
import random
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

In [2]:
class IMH(object):
    """
    Class IMH
    #########
    
    Implementation of IMH (Independent Metropolis Hasting) algorithm explained in :
    << Using parallel computation to improve Independent Metropolis Hasting based estimation >>
    Jacob and al. 2011
    
    Arguments
    ---------
    - omega (function) : density function of the distribution to simulate
    - gen_candidate (function(N) ) : reurn an array of N values gen from the candidate distribution

    - x0 (float) : initialisation
    - N (int) : length of the markov chain
    - njobs (int) : 

    Methods
    -------
    - compute_permutation : Return the permuted index 1,2,...,n according to self.permutation method
    - fit_simple : implementation of the fundamental version of the IMH algorithm 
    - fit_block : implementation of the block version of the IMH algorithm 
    - fit : main method interface
    """
    
    def __init__(self, omega, gen_candidate) :
        self.omega = np.vectorize(omega)
        self.gen_candidate = gen_candidate

    def compute_permutation(self, n):

        if self.permutation == "same_order":
            return_list = [list(range(1,n))]*self.njobs
        elif self.permutation == "random": 
            return_list=[list(np.random.permutation(n)) for i in range(self.njobs)]
        else:
            print("error : permutation must be 'same_order' or 'random' ")
        return(return_list)


    def fit_simple(self, b=0):

        n = self.y.shape[0] # either equal to : 
                            # - self.N (when method ='simple')
                            # - self.nblocks (when method ='parallel')
        
        
        i = np.random.permutation(self.nblocks)
        i = [0]+list(i)
        # go
        current = 0
        weight = np.full(fill_value = 0, shape = n)
        for candidate in range(1, len(i)):
            ratio = min(1, self.omega_y[i[candidate],b]/self.omega_y[i[current],b])
            u = np.random.binomial(1,ratio)
            current += u*(candidate-current) # current is updated to candidate if u = 1 
                                             # and stay current if u = 0

            weight[i[current]] += 1 # add current value to the chain
        
        return weight

    
    def fit_block(self):
        
        weight = np.full(fill_value = 0, shape = self.y.shape)
        with Pool(self.njobs) as p:
            for b in range(self.B): 

                weight_block = np.array(p.map(self.fit_simple, [b]*self.njobs))
                weight[:,b] = weight_block.sum(axis = 0)

                if b < self.B-1 : # init the next block picking randomly in the current block
                    self.y[0,b+1] = np.random.choice(self.y[1:,b], size=1, p= weight[1:,b]/weight[1:,b].sum())
                    self.omega_y[0,b+1] = self.omega(self.y[0,b+1])
        return weight


    def fit(self, x0, N, method = 'simple', B = 1, njobs = 1):
        self.B = B
        self.nblocks = int(N/B)
        self.njobs = njobs
        self.N = N     

        if method == "simple":            
            # (1) creation of candidate sample
            self.y = np.reshape(self.gen_candidate(self.N-1), newshape=(self.N-1,1))
            # (2) add init value
            self.y = np.append([[x0]], self.y, axis = 0)
            # (3) computation of omega values
            self.omega_y = self.omega(self.y)
            # (4) computation of weight with IMH algo
            self.weights = self.fit_simple()
            
        elif method == 'parallel':
            # (1) creation of candidate sample
            self.y = np.array(Pool(njobs).map(self.gen_candidate,[int(self.N/njobs)]*njobs))
            # (3) computation of omega values
            self.omega_y = np.array(Pool(njobs).map(self.omega, list(self.y)))
            
            # reshape : this is usefull when nblocks!=njobs
            self.y = np.reshape(self.y, newshape=(self.nblocks, self.B))
            self.omega_y = np.reshape(self.omega_y,newshape=(self.nblocks, self.B))
            # (2) add init value
            self.y = np.append(np.full((1,self.B), x0), self.y, axis = 0)
            self.omega_y = np.append(np.full((1,self.B), self.omega(x0)), self.omega_y, axis = 0)
            
            # (4) computation of weight with IMH algo
            self.weights = self.fit_block() # computation of weight
        
        self.weights = np.reshape(self.weights,newshape=self.y.shape)
        self.expectation = np.average(self.y[1:,:], weights= self.weights[1:,:])
        
        return self

On va simuler une loi normale de moyenne égale à 3. <br> 
La loi candidate sera une cauchy centrée en 3.

In [3]:
# Parameters
def omega(x):  # densité de la loi normale
    return((1+x**2)*np.exp(-(x)**2/2))

def gen_candidate(N):
    np.random.seed() # for seeding the generator
    return np.random.standard_cauchy(size = N)

In [4]:
IMH_computer  = IMH(omega, gen_candidate)

In [5]:
IMH_computer.fit(x0 = 0, N = 4000, method='simple')
print("Estimation de la moyenne avec la methode simple : ", IMH_computer.expectation)

IMH_computer.fit(x0 = 0, N = 4000, method='parallel', B=10, njobs=4)
print("Estimation de la moyenne avec la methode parallel : ", IMH_computer.expectation)

Estimation de la moyenne avec la methode simple :  0.00507782535995
Estimation de la moyenne avec la methode parallel :  -0.0232835264956


Les estimations de la moyenne semblent correct ! <br>
Evaluons maintenant les performances des deux méthodes selon 2 critères : 
- le temps d'execution
- la variances de l'estimation

**Temps d'execution :**

In [6]:
%timeit -n 10 -r 3 IMH_computer.fit(x0 = 0, N = 4000, method='simple')

45.6 ms ± 752 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [7]:
%timeit -n 3 -r 3 IMH_computer.fit(x0 = 0, N = 4000, method='parallel', B=10, njobs=4)

214 ms ± 4.27 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


**Variance :**

In [8]:
estimation_simple = []
for i in range(1000):
    IMH_computer.fit(x0 = 0, N = 40, method='simple')
    estimation_simple += [IMH_computer.expectation]
    
np.var(estimation_simple)

0.04640096912745427

In [9]:
estimation_parallel= []
for i in range(10):
    IMH_computer.fit(x0 = 0, N = 40, method='parallel', B=10, njobs=4)
    estimation_parallel += [IMH_computer.expectation]

np.var(estimation_parallel)

0.0360482433817058

A nombre *N* de candidats égaux, la méthode *parallel* donnera un estimateur de **variance plus faible** que l'estimateur obtenu avec la méthode *simple*. Néanmoins, à nombre *N* de candidats égaux, **la méthode *parallel* aura un temps d'execution un peu plus long que la méthode *simple* ** car elle execute plus de calcul. Cela se traduit les sommes des poids obtenus :

In [12]:
IMH_computer.fit(x0 = 0, N = 4000, method='parallel', B=10, njobs=4).weights.sum()

16000

In [13]:
IMH_computer.fit(x0 = 0, N = 4000, method='simple').weights.sum()

4000

La somme est plus grande pour la méthode *parallel* : plus de chaines de Markov ont été généré avec les candidats qu'avec la méthode *parallel*. D'où la variance plus petite mais le temps d'excution plus long !

Ces deux critères - variance et temps - sont liées par le paramètre *N* : en effet, plus *N* est grand, plus la variance sera faible mais plus le temps d'execution sera long. On pourrait donc utiliser la méthode parallèle pour avoir un estimateur plus rapide, mais à variance égale qu'avec la méthode simple... Reste à savoir si le gain de performance variance de la methode *parrallel* compense la perte de performance en temps ... Cela dépend grandement de l'ordinateur utilisé ! 